In [ ]:
pip --version

pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [2]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=5d426f4a1bdc74081b2cdfe54f3c2c2b6975747dfcac0786dd755ac404dc9dfa
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [84]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, sum, avg, month

# Создание SparkSession
spark = SparkSession.builder.appName("Final task").getOrCreate()

# Чтение CSV-файла
df = spark.read.csv("/content/web_server_logs.csv", header=True, inferSchema=True)
df.show(3)
df.printSchema()

df.createOrReplaceTempView("temp")
# ==========================================================================================================
# Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP.
df.groupby("ip").count().orderBy("count", ascending=False).show(10)

spark.sql("""
    select
          ip,
          count(response_code)
    from temp
    group by ip
    order by count(response_code)
    limit 10
""").show()
# ==========================================================================================================
# Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.

df.groupBy("method").count().orderBy("count", ascending=False).show()

# ==========================================================================================================
# Профильтруйте и посчитайте количество запросов с кодом ответа 404.

amount = spark.sql("""
    select
        count(1) as amount
    from temp
    where response_code = 404
""")
print(f'Number of 404 response code:{amount.collect()[0]["amount"]}')

# ==========================================================================================================
# Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.

spark.sql("""
  select
      cast(timestamp as date),
      sum(response_size) as total_response_size
  from temp
  group by cast(timestamp as date)
  order by sum(response_size) desc
  """).show()

+-------------+--------------------+------+--------------------+-------------+-------------+
|           ip|           timestamp|method|                 url|response_code|response_size|
+-------------+--------------------+------+--------------------+-------------+-------------+
| 1.100.233.90|2024-05-25 06:08:...|  POST|                list|          500|          900|
| 158.16.181.2|2024-05-23 19:40:...|  POST|      wp-content/tag|          301|         2431|
|53.89.224.199|2024-05-07 21:51:...|  POST|explore/categorie...|          404|         6401|
+-------------+--------------------+------+--------------------+-------------+-------------+
only showing top 3 rows

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

+---------------+-----+
|             ip|count|
+---------------+-----+
|  